<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/Vision_Transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import random
from google.colab import drive
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
import PIL.Image
import sys

In [ ]:
"""
# Clone the GitHub repository
!git clone https://github.com/bilmark0/Agile-Manufacturing-TDK-.git

# Append the modules path
sys.path.append("/content/Agile-Manufacturing-TDK-/CV/modules")

# Import DependencyManager from the correct module path
from dependency_manager import DependencyManager
"""

In [ ]:
!pip install kaggle
from google.colab import files
import zipfile

uploaded = files.upload()

# Move the uploaded kaggle.json file to the proper directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the training data from kaggle
!kaggle competitions download -c airbus-ship-detection

# Unzip the file and delete the zip afterwards
zip_file = '/content/airbus-ship-detection.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()
os.remove(zip_file)
if os.path.exists('/content/sample_data'):
  shutil.rmtree('/content/sample_data')

print("Download and extraction complete.")

In [ ]:
# Define source directories and target directory
source_dirs = ['Reference', 'error_2', 'error_3']
target_dir = 'training_data'

# Create the target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Move each source directory into the target directory
for dir_name in source_dirs:
    shutil.move(dir_name, target_dir)

print("Folders moved to", target_dir)

In [ ]:
data_dir = '/path/to/your/faulty_directory'
img_height, img_width = 240, 380

# Data generator with validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

# Training and validation data generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define the Vision Transformer model
def create_vit_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, (3, 3), padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Reshape((-1, x.shape[-1]))(x)  # Reshape for transformer

    # Transformer block
    for _ in range(4):  # 4 transformer blocks
        x_res = x
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
        x = layers.Add()([x_res, attention_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x_res = x
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dense(x.shape[-1])(x)
        x = layers.Add()([x_res, x])

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

# Create and compile the model
model = create_vit_model((img_height, img_width, 3))
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss}, Accuracy: {accuracy}')

# Save the trained model
model.save('/content/drive/MyDrive/object_detection_vit_model.h5')